In [ ]:
import os
from skimage.feature import hog
from skimage.io import imread
from skimage.transform import resize

from tensorflow.keras.applications import VGG16, VGG19

from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications import vgg19

from tensorflow.keras.preprocessing import image

import numpy as np

In [ ]:
# Defina os diretórios
source_dir = 'selected_images_dog_cat'

uploaded_images = []
# Percorra os arquivos no diretório de origem
for filename in os.listdir(source_dir):
    source_file = os.path.join(source_dir, filename)
    source_file = source_file.replace('\\', '/')
    uploaded_images.append(source_file)

print(uploaded_images)

In [ ]:
def extract_hog(img_size, pixel_per_cell):
    hog_features = []
    for filename in uploaded_images:
        image = imread(filename)
        image_resized = resize(image, (img_size, img_size))
        fd, hog_image = hog(image_resized, orientations=9, pixels_per_cell=(pixel_per_cell, pixel_per_cell),
                            cells_per_block=(2, 2), visualize=True, channel_axis=-1)
        hog_features.append(fd)
    return hog_features

In [ ]:
hog_128_16 = extract_hog(128, 16)
hog_128_20 = extract_hog(128, 20)
hog_256_16 = extract_hog(256, 16)
hog_256_20 = extract_hog(256, 20)

In [ ]:
def extract_cnn(img_size, model, vgg):
    cnn_features = []
    for filename in uploaded_images:
        imagem = imread(filename)
        ##print(imagem.shape)
        print(filename)
        imagem_reduzida = resize(imagem, (img_size, img_size))
        x = image.img_to_array(imagem_reduzida)
        x = np.expand_dims(x, axis=0)
        x = vgg.preprocess_input(x)
        features = model.predict(x)
        features_flatten_vgg = features.flatten()
        cnn_features.append(features_flatten_vgg)
    return cnn_features


In [ ]:
model_16_avg = VGG16(weights='imagenet', include_top=False, pooling='avg')
model_16_max = VGG16(weights='imagenet', include_top=False, pooling='max')
model_19_avg = VGG19(weights='imagenet', include_top=False, pooling='avg')
model_19_max = VGG19(weights='imagenet', include_top=False, pooling='max')

cnn_16_avg_128 = extract_cnn(128, model_16_avg, vgg16)
cnn_16_max_128 = extract_cnn(128, model_16_max, vgg16)

cnn_16_avg_256 = extract_cnn(256, model_16_avg, vgg16)
cnn_16_max_256 = extract_cnn(256, model_16_max, vgg16)

cnn_19_avg_128 = extract_cnn(128, model_19_avg, vgg19)
cnn_19_max_128 = extract_cnn(128, model_19_max, vgg19)

cnn_19_avg_256 = extract_cnn(256, model_19_avg, vgg19)
cnn_19_max_256 = extract_cnn(256, model_19_max, vgg19)

# print(cnn_16_avg_128)

In [ ]:
features_dict = {
    'hog_128_16': hog_128_16,
    'hog_128_20': hog_128_20,
    'hog_256_16': hog_256_16,
    'hog_256_20': hog_256_20,
    'cnn_16_avg_128': cnn_16_avg_128,
    'cnn_16_max_128': cnn_16_max_128,
    'cnn_16_avg_256': cnn_16_avg_256,
    'cnn_16_max_256': cnn_16_max_256,
    'cnn_19_avg_128': cnn_19_avg_128,
    'cnn_19_max_128': cnn_19_max_128,
    'cnn_19_avg_256': cnn_19_avg_256,
    'cnn_19_max_256': cnn_19_max_256
}

In [ ]:
import json

# Function to convert numpy arrays to lists in the dictionary
def convert_ndarrays(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {k: convert_ndarrays(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_ndarrays(i) for i in obj]
    return obj

# Convert all ndarrays to lists
serializable_features_dict = convert_ndarrays(features_dict)

# Write the dictionary to JSON
with open('features.json', 'w') as f:
    json.dump(serializable_features_dict, f, indent=4)

print("Data saved to features.json")
